In [ ]:
Importing the different libraries:

In [1]:
import pandas as pd
import numpy as np
import re  #For managing the regular expession--> a filter for text
from sklearn.feature_extraction.text import CountVectorizer # this library is used to transform the text to a vector (bags of words)
from sklearn.model_selection import train_test_split   #to divide the dataset in training and test set
from sklearn.linear_model import LogisticRegression  #to utilize the logistic regression
from sklearn.pipeline import Pipeline   # Pileline implementation
from sklearn.feature_extraction.text import TfidfVectorizer  #To work with tf.idf term weight
import nltk  #This libraty is builded to work with token/stems 
nltk.download("punkt") #as above
from sklearn import tree
from sklearn.metrics import confusion_matrix  #To check the accuracy->confusion matrix
from sklearn.model_selection import cross_val_score  #To work with cross validation
from sklearn.ensemble import RandomForestClassifier #Random Forest CLassifier
from sklearn.naive_bayes import MultinomialNB #Multinomial naive Bayer Classifier
import xgboost as xgb #XgBoost Classifier
r_state=42

[nltk_data] Downloading package punkt to C:\Users\nabu-
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


We create some definitions for cleaning (second part):

In [2]:
def pulizia(item):
    for i in item:
        item = re.sub("\d+", " ", item) #It replaces the numbers with empty values
        item= re.sub("\W+", " ", item) #It replaces special caracters with an empty value (non alphanumeric character)
    return item

In [3]:
def clean(item):
    for i in item:
        item = re.sub("\D+", " ", item) #It replaces any non-digit character (alpha + special character) with an empty value 
    return item

In [4]:
def delete(item):
    item = item.lower().split()
    cancellare = ['codaf','sabina','elisena martina','marzia','e','sig ra','marzia e','erica','marco','sergio','r','urgente','doris','sig ','marchetti','moreno','federica','richimare','ombreatta','simonetta','stura','milena','(',')','anariten','dtb' ,'x','per','helpdesk', '/', 'richiamare', 'carmela','paola' , 'alessandra' , 'francesco','francesca' , 'giorgia' , 'claudia' , 'paolo' , 'cristina' , 'giuseppe' , 'caterina' , 'carmela' , 'barbara' , 'antonio' , 'michela' , 'elena' , 'ddt' , 'simone' , 'emanuela' , 'silvia' , 'claudio' , 'roberto' , 'laura' , 'massimo' , 'romina' , 'mirella' , 'marco' , 'sara' , 'lara' , 'roberta' , 'davide' , 'mauro' , 'maria' , 'gianni' , 'chiara' , 'rita' , 'sabrina' , 'gabriella' , 'vanessa' , 'lucia' , 'giovanna' , 'maila' , 'andrea' , 'catia' , 'marusca' , 'gazzi' , 'marina' , 'ariadna' , 'monica' , 'giuliano' , 'vanna' , 'ottaviani' , 'saloli' , 'irene' , 'guzzaroni' , 'tiziana' , 'manuela' , 'arianna' , 'marianna' , 'lube' , 'stefano' , 'giuseppina' , 'teresa' , 'ylenia' , 'luciano' , 'nadia' , 'grazia' , 'ciao' , 'sele' , 'katia' , 'camilletti' , 'daniela' , 'srl' , 'defoot' , 'luca' , 'brogliaccio' , 'elvio' , 'brunori' , 'patrizia' , 'marica' , 'sonia' , 'antonella' , 'lorenzo' , 'nicola' , 'giacomo' , 'noemi' , 'tonino' , 'franco' , 'battistoni' , 'mosciatti' , 'fabrizio' , 'david' , 'michele' , 'giovanni' , 'cinzia' , 'valentina' , 'desc' , 'merlonni' , 'elisabetta' , 'angela' , 'isabella' , 'sandro' , 'ilaria' , 'danila' , 'elisa' , 'miriam' , 'orietta' , 'daniele' , 'stefania' , 'adria' , 'collega' , 'matteo' , 'leonardo' , 'carmelica' , 'marika' , 'elisena' , 'alberto' , 'emanuele' , 'raffaele' , 'gianfranco' , 'letizia' , 'tania' , 'denis' , 'gabriele' , '' , 'enasarco' , 'ester' , 'silvana' , 'guido' , 'mariarita' , 'raffaella' , 'walter' , 'mara' , 'linneo' , 'ivano' , 'ghergo' , 'luigi' , 'giulia' , 'vittorio' , 'san' , 'fabio' , 'bacchiani' , 'luciana' , 'giusi' , 'mirian' , 'lorena' , 'baiardi' , 'sirolo' , 'mario' , 'corvatta' , 'alessandro' , 'giogia' , 'urgente' , 'helpdesk' , 'pezzella' , 'chiarimenti' , '' , 'saloli' , 'guazzaroni' , 'lube' , 'med' , 'giuseppina' , 'ylenia' , 'mrp' , 'grazia' , 'katia' , 'camilletti' , 'daniela' , 'sedoor' , 'brunori' , 'petriccione' , 'tonino' , 'mosciatti' ,'donatella','moreno', 'david' , 'irci' , 'merloni' , 'anusca' , 'urgentemente','lupini','antonelli']
    for i in cancellare:
        while i in item:
            variable = item.index(i)
            item.pop(variable)
    item=' '.join(map(str, item))
    item=str(item)
    return item

In [5]:
def StringToInt(item):
    item = int(item)
    return item

# Task1: Sentiment Analysis
### Scope: We need to do sentiment analysis on a precise column (Descrizione_Risolutore). In this way we obtain another column (SA) with positive/negative label according to the previous column.

Importing CSV and working a little on cleaning (NUll values):

In [6]:
df = pd.read_csv(r"C:\Users\nabu-pc\Desktop\Data Science\Project field\labelHani.csv",sep=";",encoding ="unicode_escape")

In [7]:
df.head(2)

,Unnamed: 0,durata_risoluzione,Cliente,Utente_Preso_chiamata,Utente_Che_ha_risolto,Data_Arrivo_chiamata,Descrizione_Chiamata,Classe,Natura,Nome_Programma_Inerente_Chiamata,Origine_Proglema,ID_CHIAMATA,Sentiment Analysis,Descrizione_Risolutore,Data_Chiusura_Chiamata,Data_Gestione_Chiamata,newcall,label,Unnamed: 18
0,9572,56,22,34,34,17/11/2016 17:21,Per HelpDesk / Richiamare Paolo x Comessa,SW,sw,NaN,Configurazione,67398,Pos,Fatto Query 9. per trovare ore fatturate da qu...,17/11/2016 18:18,giovedì 17 novembre 2016,comessa,commessa,NaN
1,10149,3,1632,62,34,15/11/2017 11:35,Per HelpDesk / Richiamare Sig. Tagliafierro\n\...,SW,sw,Contabile,Cliente,71129,Neg,Non riesco a contattare il numero telefonico 0...,10/04/2018 15:09,giovedì 16 novembre 2017,sig tagliafierro registrazione nota credito te...,mastrino,NaN


In [8]:
df = df.dropna(axis=0, subset=['Descrizione_Risolutore'])    #We need to clean the "Descrizione_Risolutore" column from NULL values

In [9]:
len(df)

646

In [10]:
df.rename(columns={'Sentiment Analysis':'SA'}, inplace=True)

In [11]:
df["SA"].value_counts()  # the classes are balanced!!

Pos    490
Neg    156
Name: SA, dtype: int64

Classification with a Supervised Traning Model: model1 (Vector + Logistic Regression)

In [12]:

X_train, X_test, y_train, y_test = train_test_split(df.Descrizione_Risolutore, df.SA, random_state=42)

In [13]:
y_train.value_counts()  #The training set is balanced!!

Pos    374
Neg    110
Name: SA, dtype: int64

In [14]:
y_test.value_counts()   #The test set is balanced!!!

Pos    116
Neg     46
Name: SA, dtype: int64

In [15]:
model1 = Pipeline([                           # I create a pipeline = at the same time create a vector and apply a logistic classifier
    ("vectorizer", TfidfVectorizer()),
    ("classifier", LogisticRegression())
])

In [16]:
model1.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [17]:
score1 = model1.score(X_test, y_test)

In [18]:
score1

0.75308641975308643

In [19]:
pred=model1.predict(X_test)

In [20]:
cm_df = pd.DataFrame(confusion_matrix(np.asarray(y_test), pred).T, index=model1.classes_,
                     columns=model1.classes_)
print(cm_df)

     Neg  Pos
Neg    7    1
Pos   39  115


Improving our model with stemming :

In [21]:
ps = nltk.stem.PorterStemmer()

In [22]:
def tokenize_with_stemming(text):     # setting a function to get stems from a text
    return [ps.stem(token) for token
        in nltk.tokenize.word_tokenize(text)]

In [23]:
tokenize_with_stemming("pino pinetto pinotto ara")

['pino', 'pinetto', 'pinotto', 'ara']

In [24]:
model2 = Pipeline([
    ("vectorizer", TfidfVectorizer(min_df=3, tokenizer=tokenize_with_stemming)),
    ("classifier", LogisticRegression())
])

In [25]:
model2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [26]:
score2 = model2.score(X_test, y_test)

In [27]:
score2

0.77160493827160492

In [28]:
pred=model2.predict(X_test)

In [29]:
cm_df = pd.DataFrame(confusion_matrix(np.asarray(y_test), pred).T, index=model2.classes_,
                     columns=model2.classes_)
print(cm_df)

     Neg  Pos
Neg   10    1
Pos   36  115


Improving our model with bigrams and max features(165):

In [30]:
model3 = Pipeline([
    ("vectorizer", TfidfVectorizer(min_df=3, ngram_range=(1, 2), tokenizer=tokenize_with_stemming,max_features=165)),
    ("classifier", LogisticRegression())
])

In [31]:
model3.fit(X_train, y_train);

In [32]:
score3 = model3.score(X_test, y_test)  

In [33]:
score3

0.80246913580246915

In [34]:
pred=model3.predict(X_test)

In [35]:
cm_df = pd.DataFrame(confusion_matrix(np.asarray(y_test), pred).T, index=model3.classes_,
                     columns=model3.classes_)
print(cm_df)

     Neg  Pos
Neg   15    1
Pos   31  115


Try with cross-validation method:

In [36]:
scores = cross_val_score(model3, df.Descrizione_Risolutore, df.SA, scoring='accuracy', cv = 10)
average_accuracy = np.mean(scores) * 100
print("The average accuracy computed with Cross Validation is {0:.2f}%".format(average_accuracy))  #Cross-validation improve the result!!!

The average accuracy computed with Cross Validation is 84.38%


Trying with random Forest!!!

In [37]:
model4 = Pipeline([
    ("vectorizer", TfidfVectorizer(min_df=3, ngram_range=(1, 2),tokenizer=tokenize_with_stemming,max_features=165)),
    ("classifier", RandomForestClassifier(n_estimators=500))
])

In [38]:
model4.fit(X_train, y_train);

In [39]:
score4 = model4.score(X_test, y_test)

In [40]:
score4

0.84567901234567899

In [41]:
pred=model4.predict(X_test)

In [42]:
cm_df = pd.DataFrame(confusion_matrix(np.asarray(y_test), pred).T, index=model4.classes_,
                     columns=model4.classes_)
print(cm_df)

     Neg  Pos
Neg   22    1
Pos   24  115


In [43]:
cross_val_score(model4, df.Descrizione_Risolutore, df.SA, scoring='accuracy', cv = 10)
average_accuracy = np.mean(scores) * 100
print("The average accuracy computed with Cross Validation is {0:.2f}%".format(average_accuracy)) #Cross validation doesn't work well on Random Forest

The average accuracy computed with Cross Validation is 84.38%


Trying with Multinomial Bayes Classifier:

In [44]:
model5 = Pipeline([
    ("vectorizer", TfidfVectorizer(min_df=3, ngram_range=(1, 2),tokenizer=tokenize_with_stemming,max_features=165)),
    ("classifier", MultinomialNB())
])

In [45]:
model5.fit(X_train, y_train);

In [46]:
score5 = model5.score(X_test, y_test)

In [47]:
score5

0.80246913580246915

In [48]:
pred=model5.predict(X_test)

In [49]:
cm_df = pd.DataFrame(confusion_matrix(np.asarray(y_test), pred).T, index=model5.classes_,
                     columns=model5.classes_)
print(cm_df)

     Neg  Pos
Neg   14    0
Pos   32  116


Trying with XGBoost:

In [50]:
model6 = Pipeline([
    ("vectorizer", TfidfVectorizer(min_df=3, ngram_range=(1, 2),tokenizer=tokenize_with_stemming,max_features=165)),
    ("classifier", xgb.XGBClassifier())
])

In [51]:
model6.fit(X_train, y_train);

In [52]:
score6 = model6.score(X_test, y_test)

In [53]:
score6

0.81481481481481477

In [54]:
pred=model6.predict(X_test)

In [55]:
cm_df = pd.DataFrame(confusion_matrix(np.asarray(y_test), pred).T, index=model6.classes_,
                     columns=model6.classes_)
print(cm_df)

     Neg  Pos
Neg   18    2
Pos   28  114


In [ ]:
A summary of our models:

In [56]:
listScore = [score1,score2,score3,score4,score5,score6]

In [57]:
listModel = ["TfIdf + Logi","TdIdf(tokens/stemming) + Logi","TdIdf(tokens/stemming/ngrams/maxfeatures) + Logi","TdIdf(tokens/stemming/ngrams/maxfeatures) + RandomForest","TdIdf(tokens/stemming/ngrams/maxfeatures) + MultinomialBC","TdIdf(tokens/stemming/ngrams/maxfeatures) + XGBoost"]

In [60]:
SummaryAcc = pd.DataFrame({"Model": listModel, "Accuracy": listScore})
SummaryAcc = SummaryAcc.iloc[:, ::-1]
SummaryAcc.style.set_properties(subset=['Model'], **{'width': '345px'})

,Model,Accuracy
0,TfIdf + Logi,0.753086
1,TdIdf(tokens/stemming) + Logi,0.771605
2,TdIdf(tokens/stemming/ngrams/maxfeatures) + Logi,0.802469
3,TdIdf(tokens/stemming/ngrams/maxfeatures) + RandomForest,0.845679
4,TdIdf(tokens/stemming/ngrams/maxfeatures) + MultinomialBC,0.802469
5,TdIdf(tokens/stemming/ngrams/maxfeatures) + XGBoost,0.814815


We estimate the labels on the whole dataset with the best model (TdIdf(tokens/stemming/ngrams/maxfeatures) + RandomForest) !!!

In [291]:
model = Pipeline([
    ("vectorizer", TfidfVectorizer(min_df=3, ngram_range=(1, 2),tokenizer=tokenize_with_stemming,max_features=165)),
    ("classifier", RandomForestClassifier(n_estimators=500))
])

In [290]:
dfAll = pd.read_excel(r"C:\Users\nabu-pc\Desktop\Data Science\Project field\ProjFieldBase.xlsx")

In [292]:
model.fit(df.Descrizione_Risolutore, df.SA); #I train the model on the all dataset with labels

In [279]:
dfAll['cleancall']=dfAll['Descrizione_Chiamata'].apply(delete)  #We delete symbol/numbers from "Descrizione chiamata"
dfAll['newcall']=dfAll['cleancall'].apply(pulizia) # We delete some useless words (mainly names)

In [315]:
dfAll['ID2']=dfAll['ID'].apply(clean)

In [316]:
dfAll.head(50)

,durata_risoluzione,Data_Arrivo_chiamata,Descrizione_Chiamata,Classe,Natura,Origine_Proglema,Descrizione_Risolutore,Data_Chiusura_Chiamata,Data_Gestione_Chiamata,CLIENTE,UTENTE,ID,DURATA SEGN.,SA,Topic,ID2
0,1,2016-01-14 10:39:22,Per HelpDesk / Richiamare Carmela,SW,sw,Cliente,Supporto per ripristinare protocolli autofatture,2016-01-14 10:40:49,giovedì 14 gennaio 2016,C1836,U34,ID64898,0,Pos,altro,64898
1,1,2016-01-18 12:56:18,Per HelpDesk / Richiamare Maila x Ammortamenti,SW,sw,Cliente,"Supporto per gestire beni inferiori a 516,00 euro",2016-02-22 15:26:32,lunedì 18 gennaio 2016,C233,U34,ID64926,35,Pos,contabilitÃ,64926
2,1,2016-01-21 10:03:48,Per HelpDesk / Richiamare Francesca per fattur...,SW,sw,Cliente,Sbloccato fatturazione Killato provesso,2016-01-21 10:04:49,giovedì 21 gennaio 2016,C1887,U34,ID64962,0,Pos,fattura,64962
3,1,2016-01-20 17:47:46,Per HelpDesk / Richiamare Dtb anariten (come f...,SW,sw,Cliente,Non si può fare.,2016-01-21 18:03:57,giovedì 21 gennaio 2016,C2182,U34,ID64958,1,Pos,altro,64958
5,1,2016-01-28 10:00:41,Per HelpDesk / Richiamare Correggere movimento,SW,sw,Cliente,Coretto movimento contabile anno 2015,2016-01-28 10:28:39,giovedì 28 gennaio 2016,C2182,U34,ID64997,0,Pos,altro,64997
6,1,2016-02-04 15:03:52,Per HelpDesk / Richiamare Alessandra,SW,sw,Cliente,Supporto per mettere a menu Stipendi.c,2016-02-04 15:28:05,giovedì 4 febbraio 2016,C1711,U34,ID65064,0,Pos,altro,65064
7,1,2016-02-04 15:23:15,Per HelpDesk / Richiamare Francesca x iva veng...,SW,sw,Cliente,Spiegato a Barbara come deve operare,2016-02-08 16:00:54,lunedì 8 febbraio 2016,C1582,U34,ID65066,4,Pos,iva,65066
8,1,2016-02-08 16:31:09,Per HelpDesk / Richiamare Fabio per pulizia ar...,SW,sw,Cliente,Fabio non c'è,2016-02-18 11:44:11,lunedì 8 febbraio 2016,C2030,U34,ID65090,10,Neg,inventario,65090
9,1,2016-02-22 08:40:25,Per HelpDesk / Richiamare x Liquidazione iva,SW,sw,Cliente,Faccio intervento Domani,2016-02-22 15:23:27,lunedì 22 febbraio 2016,C2182,U34,ID65199,0,Neg,iva,65199
10,1,2016-02-29 16:09:49,Per HelpDesk / Richiamare X Articoli Obsoleti,SW,sw,Cliente,Fatto preventivo,2016-03-03 12:55:28,giovedì 3 marzo 2016,C510,U34,ID65275,3,Pos,inventario,65275


In [294]:
dfAll = dfAll.dropna(axis=0, subset=['Descrizione_Risolutore'])

In [295]:
len(dfAll)

10210

In [296]:
dfAll['SA'] = model.predict(dfAll.Descrizione_Risolutore)

In [297]:
dfAll.head(10)

,durata_risoluzione,Data_Arrivo_chiamata,Descrizione_Chiamata,Classe,Natura,Nome_Programma_Inerente_Chiamata,Origine_Proglema,Descrizione_Risolutore,Data_Chiusura_Chiamata,Data_Gestione_Chiamata,CLIENTE,UTENTE,ID,DURATA SEGN.,SA
0,1,2016-01-14 10:39:22,Per HelpDesk / Richiamare Carmela,SW,sw,NaN,Cliente,Supporto per ripristinare protocolli autofatture,2016-01-14 10:40:49,giovedì 14 gennaio 2016,C1836,U34,ID64898,0,Pos
1,1,2016-01-18 12:56:18,Per HelpDesk / Richiamare Maila x Ammortamenti,SW,sw,NaN,Cliente,"Supporto per gestire beni inferiori a 516,00 euro",2016-02-22 15:26:32,lunedì 18 gennaio 2016,C233,U34,ID64926,35,Pos
2,1,2016-01-21 10:03:48,Per HelpDesk / Richiamare Francesca per fattur...,SW,sw,NaN,Cliente,Sbloccato fatturazione Killato provesso,2016-01-21 10:04:49,giovedì 21 gennaio 2016,C1887,U34,ID64962,0,Pos
3,1,2016-01-20 17:47:46,Per HelpDesk / Richiamare Dtb anariten (come f...,SW,sw,NaN,Cliente,Non si può fare.,2016-01-21 18:03:57,giovedì 21 gennaio 2016,C2182,U34,ID64958,1,Pos
5,1,2016-01-28 10:00:41,Per HelpDesk / Richiamare Correggere movimento,SW,sw,NaN,Cliente,Coretto movimento contabile anno 2015,2016-01-28 10:28:39,giovedì 28 gennaio 2016,C2182,U34,ID64997,0,Pos
6,1,2016-02-04 15:03:52,Per HelpDesk / Richiamare Alessandra,SW,sw,NaN,Cliente,Supporto per mettere a menu Stipendi.c,2016-02-04 15:28:05,giovedì 4 febbraio 2016,C1711,U34,ID65064,0,Pos
7,1,2016-02-04 15:23:15,Per HelpDesk / Richiamare Francesca x iva veng...,SW,sw,NaN,Cliente,Spiegato a Barbara come deve operare,2016-02-08 16:00:54,lunedì 8 febbraio 2016,C1582,U34,ID65066,4,Pos
8,1,2016-02-08 16:31:09,Per HelpDesk / Richiamare Fabio per pulizia ar...,SW,sw,NaN,Cliente,Fabio non c'è,2016-02-18 11:44:11,lunedì 8 febbraio 2016,C2030,U34,ID65090,10,Neg
9,1,2016-02-22 08:40:25,Per HelpDesk / Richiamare x Liquidazione iva,SW,sw,NaN,Cliente,Faccio intervento Domani,2016-02-22 15:23:27,lunedì 22 febbraio 2016,C2182,U34,ID65199,0,Neg
10,1,2016-02-29 16:09:49,Per HelpDesk / Richiamare X Articoli Obsoleti,SW,sw,NaN,Cliente,Fatto preventivo,2016-03-03 12:55:28,giovedì 3 marzo 2016,C510,U34,ID65275,3,Pos


# Task 2: Topic creation

### Scope: We need to create relevant topics for newcall column. In this column we have a problem description (text) for each call.

Importing data:

In [243]:
ag = pd.read_csv(r"C:\Users\nabu-pc\Desktop\Data Science\Project field\label17.csv",sep=",",encoding ="unicode_escape")

In [244]:
X= ag['newcall']
y= ag['label']

Tryning with Decision Tree

In [245]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= r_state)# default Train 0.75- Test 0.25
print("There are {} samples in the training dataset".format(X_train.shape[0]))
print("There are {} samples in the testing dataset".format(X_test.shape[0]))

There are 479 samples in the training dataset
There are 160 samples in the testing dataset


In [247]:
modelk0 = Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("classifier", tree.DecisionTreeClassifier(random_state = r_state))
])

In [248]:
modelk0.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...        min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'))])

In [249]:
scorek0=modelk0.score(X_test,y_test)

Trying with RandomForest:

In [250]:
modelk1 = Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("classifier", RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
            oob_score=False, random_state=r_state, verbose=0,
            warm_start=False) )
])

In [251]:
modelk1.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...timators=150, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [252]:
scorek1=modelk1.score(X_test, y_test)

In [253]:
modelk2= Pipeline([
    ("vectorizer", TfidfVectorizer(binary=True,min_df=3, ngram_range=(1, 3),tokenizer=tokenize_with_stemming,max_features=400)),
    ("classifier", RandomForestClassifier(n_estimators=150, random_state=r_state,criterion='entropy',verbose=0.))
])

In [254]:
modelk2.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=400, min_df=3,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True...n_jobs=1,
            oob_score=False, random_state=42, verbose=0.0,
            warm_start=False))])

In [255]:
scorek2=modelk2.score(X_test, y_test)

Trying with XGBoost:

In [256]:
modelk3 = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=tokenize_with_stemming,binary=True,min_df=3, ngram_range=(1, 2))),
    ("classifier", xgb.XGBClassifier(max_feature=0,random_state=r_state,n_estimators=950))
])

In [257]:
modelk3.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=Tru... reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1))])

In [258]:
scorek3=modelk3.score(X_test, y_test)

Trying with MultinominalNB:

In [259]:
modelk4 = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=tokenize_with_stemming,binary=True,min_df=3, ngram_range=(1, 2),max_features=400)),
    ("classifier", MultinomialNB())
])

In [260]:
modelk4.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=400, min_df=3,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True...True, vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [261]:
scorek4=modelk4.score(X_test, y_test)

In [262]:
listScorek = [scorek0,scorek1,scorek2,scorek3,scorek4]

In [236]:
listModelk = ["Tree","Random Forest","TdIdf(tokens/stemming/ngrams/maxfeatures) + RandomForest","TdIdf(tokens/stemming/ngrams/maxfeatures) + XGBoost "," TdIdf(tokens/stemming/ngrams/maxfeatures) + MultinomialBC "]

In [263]:
SummaryAcc = pd.DataFrame({"Model": listModelk, "Accuracy": listScorek})
SummaryAcc = SummaryAcc.iloc[:, ::-1]
SummaryAcc.style.set_properties(subset=['Model'], **{'width': '345px'})

,Model,Accuracy
0,Tree,0.68125
1,Random Forest,0.7125
2,TdIdf(tokens/stemming/ngrams/maxfeatures) + RandomForest,0.74375
3,TdIdf(tokens/stemming/ngrams/maxfeatures) + XGBoost,0.69375
4,TdIdf(tokens/stemming/ngrams/maxfeatures) + MultinomialBC,0.65625


We estimate the labels on the whole dataset with the best model (TdIdf(tokens/stemming/ngrams/maxfeatures) + RandomForest)  !!! 

In [298]:
dfAll = dfAll.dropna(axis=0, subset=['Descrizione_Chiamata'])

In [299]:
len(dfAll)

10210

In [300]:
dfAll['Topic'] = modelk2.predict(dfAll.Descrizione_Chiamata)

In [304]:
dfAll = dfAll.drop(columns=['Nome_Programma_Inerente_Chiamata'])

In [321]:
dfAll['ID']= dfAll['ID2']

In [323]:
dfAll = dfAll.drop(columns=['ID2'])

In [327]:
dfAll.head(5)

,durata_risoluzione,Data_Arrivo_chiamata,Descrizione_Chiamata,Classe,Natura,Origine_Proglema,Descrizione_Risolutore,Data_Chiusura_Chiamata,Data_Gestione_Chiamata,CLIENTE,UTENTE,DURATA SEGN.,SA,Topic,ID
0,1,2016-01-14 10:39:22,Per HelpDesk / Richiamare Carmela,SW,sw,Cliente,Supporto per ripristinare protocolli autofatture,2016-01-14 10:40:49,giovedì 14 gennaio 2016,C1836,U34,0,Pos,altro,64898
1,1,2016-01-18 12:56:18,Per HelpDesk / Richiamare Maila x Ammortamenti,SW,sw,Cliente,"Supporto per gestire beni inferiori a 516,00 euro",2016-02-22 15:26:32,lunedì 18 gennaio 2016,C233,U34,35,Pos,contabilitÃ,64926
2,1,2016-01-21 10:03:48,Per HelpDesk / Richiamare Francesca per fattur...,SW,sw,Cliente,Sbloccato fatturazione Killato provesso,2016-01-21 10:04:49,giovedì 21 gennaio 2016,C1887,U34,0,Pos,fattura,64962
3,1,2016-01-20 17:47:46,Per HelpDesk / Richiamare Dtb anariten (come f...,SW,sw,Cliente,Non si può fare.,2016-01-21 18:03:57,giovedì 21 gennaio 2016,C2182,U34,1,Pos,altro,64958
5,1,2016-01-28 10:00:41,Per HelpDesk / Richiamare Correggere movimento,SW,sw,Cliente,Coretto movimento contabile anno 2015,2016-01-28 10:28:39,giovedì 28 gennaio 2016,C2182,U34,0,Pos,altro,64997


In [328]:
 dfAll.dropna(inplace=True)

In [330]:
dfAll['ID']=dfAll['ID'].apply(StringToInt)

In [336]:
dfAll['CLIENTE'] = dfAll['CLIENTE'].apply(clean)

In [337]:
dfAll['UTENTE'] = dfAll['UTENTE'].apply(clean)

In [338]:
dfAll.head(5)

,durata_risoluzione,Data_Arrivo_chiamata,Descrizione_Chiamata,Classe,Natura,Origine_Proglema,Descrizione_Risolutore,Data_Chiusura_Chiamata,Data_Gestione_Chiamata,CLIENTE,UTENTE,DURATA SEGN.,SA,Topic,ID
0,1,2016-01-14 10:39:22,Per HelpDesk / Richiamare Carmela,SW,sw,Cliente,Supporto per ripristinare protocolli autofatture,2016-01-14 10:40:49,giovedì 14 gennaio 2016,1836,34,0,Pos,altro,64898
1,1,2016-01-18 12:56:18,Per HelpDesk / Richiamare Maila x Ammortamenti,SW,sw,Cliente,"Supporto per gestire beni inferiori a 516,00 euro",2016-02-22 15:26:32,lunedì 18 gennaio 2016,233,34,35,Pos,contabilitÃ,64926
2,1,2016-01-21 10:03:48,Per HelpDesk / Richiamare Francesca per fattur...,SW,sw,Cliente,Sbloccato fatturazione Killato provesso,2016-01-21 10:04:49,giovedì 21 gennaio 2016,1887,34,0,Pos,fattura,64962
3,1,2016-01-20 17:47:46,Per HelpDesk / Richiamare Dtb anariten (come f...,SW,sw,Cliente,Non si può fare.,2016-01-21 18:03:57,giovedì 21 gennaio 2016,2182,34,1,Pos,altro,64958
5,1,2016-01-28 10:00:41,Per HelpDesk / Richiamare Correggere movimento,SW,sw,Cliente,Coretto movimento contabile anno 2015,2016-01-28 10:28:39,giovedì 28 gennaio 2016,2182,34,0,Pos,altro,64997


In [340]:
writer = pd.ExcelWriter('dfFinal.xlsx')
dfAll.to_excel(writer,'Sheet1')
writer.save()